# Creación de datos en sqlite3

## Importar librerías

In [1]:
import sqlite3
import pandas as pd
from faker import Faker
import unicodedata
import random

## Creación de lista con datos personales para 50000 estudiantes

In [6]:
#función para quitar las tildes de los apellidos

def quitar_tildes(texto):
    texto_normalizado = unicodedata.normalize("NFD", texto)
    #NFD significa Normal Form Decomposition, separa la vocal y la tilde en dos caracteres diferentes
    texto_sin_tildes = ''.join(
        c for c in texto_normalizado if unicodedata.category(c) != 'Mn'
        #unicodedata.category(c) devuelve una categoría Unicode para el carácter c, la tilde toma el valor de Mn
    )
    return texto_sin_tildes

fk = Faker('es_ES')
data = []
domain = ['yahoo.com','hotmail.com','gmail.com', 'outlook.com','soho.com']
for x in range(1,50001):
    first_name = fk.first_name()
    last_name = fk.last_name()
    name = first_name + " "+last_name
    email = first_name[0].lower()+quitar_tildes(last_name.lower())+"@"+domain[random.randint(0,len(domain)-1)]
    data.append((x,name,email))

## Creación de la base de datos "students.db" y la table "personal_data"

In [7]:
conn = sqlite3.connect('students.db')
cursor = conn.cursor()
# Crear la tabla 'personal_data'
cursor.execute("""
CREATE TABLE IF NOT EXISTS personal_data (
    ALUMNOID INTEGER PRIMARY KEY AUTOINCREMENT,
    NOMBRE TEXT NOT NULL,
    CORREO TEXT NOT NULL);
""")

## Insertar datos en la tabla "personal_data"

In [8]:
cursor.executemany("""
INSERT INTO personal_data (AlumnoID, nombre, correo)
VALUES (?, ?, ?);
""", data)

# Confirmar los cambios y cerrar la conexión
conn.commit()
conn.close()



## Visualización de datos

In [9]:
conn = sqlite3.connect('students.db')
df = pd.read_sql_query('SELECT * from personal_data', conn)
conn.close()
df.head()

,ALUMNOID,NOMBRE,CORREO
0,1,Itziar Torrens,itorrens@outlook.com
1,2,Patricia Durán,pduran@soho.com
2,3,Gisela Bonet,gbonet@yahoo.com
3,4,Sabina Feliu,sfeliu@gmail.com
4,5,Visitación Martin,vmartin@soho.com
